In [1]:
import numpy as np
import tensorflow as tf
import import_ipynb
from params import model_params
p = model_params('config.ini')

def freq2erb (freq_hz):
    return 9.265 * np.log(1 + freq_hz / (24.7 * 9.265))

def erb2freq (n_erb):
    return 24.7 * 9.265 * (np.exp(n_erb / 9.265) -1)

def ERBBand(N, high_lim, NFFT):
    freqRangePerBin = high_lim / (NFFT/2)
    erb_low = freq2erb(0)
    erb_high = freq2erb(high_lim)
    erb_lims = np.linspace(erb_low, erb_high, N)

    cutoffs = [erb2freq(erb_lims[i]) for i in range (N)]

    nfftborder = [np.round((cutoffs[i]+(freqRangePerBin/2))/freqRangePerBin) for i in range (N)]

    for i in range (N-2):
        if (nfftborder[i+2] - nfftborder[i]) < 2:
            nfftborder[i+2] += (2-(nfftborder[i+2] - nfftborder[i]))

    return nfftborder


def ERB_pro_matrix(nfftborder, NFFT, mode):
    N                 = len(nfftborder)
    ERB_Matrix        = np.zeros((int(NFFT / 2 +1) , N))
    
    for i in range(N - 1):
        band_size        = nfftborder[i+1]-nfftborder[i]
        
        for j in range(int(band_size)):
            ERB_Matrix[int(nfftborder[i])+j, i]  = 1 - j / band_size
            ERB_Matrix[int(nfftborder[i])+j,i+1] = j / band_size
    if mode==0:
        ERB_Matrix[:, 0]   *=2
        ERB_Matrix[:, N-1] *=2

    return ERB_Matrix

importing Jupyter notebook from params.ipynb


In [2]:
X = np.ones(481)
ERBB = ERBBand(N=32, high_lim=24000, NFFT=960)
ERBB

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 6.0,
 7.0,
 9.0,
 11.0,
 14.0,
 17.0,
 20.0,
 24.0,
 28.0,
 34.0,
 40.0,
 47.0,
 55.0,
 65.0,
 76.0,
 89.0,
 104.0,
 121.0,
 141.0,
 165.0,
 192.0,
 224.0,
 261.0,
 305.0,
 355.0,
 413.0,
 481.0]

In [9]:
ERB_Matrix = ERB_pro_matrix(ERBB, NFFT=960, mode=0) #  ERB convert matrix
Shane_B = np.sqrt(X @ ERB_Matrix)
Shane_B

array([1.41421356, 1.        , 1.        , 1.        , 1.        ,
       1.22474487, 1.41421356, 1.41421356, 1.41421356, 1.58113883,
       1.73205081, 1.87082869, 2.12132034, 2.23606798, 2.34520788,
       2.54950976, 2.73861279, 3.        , 3.24037035, 3.46410162,
       3.74165739, 4.0620192 , 4.35889894, 4.69041576, 5.04975247,
       5.43139025, 5.87367006, 6.32455532, 6.81909085, 7.34846923,
       7.93725393, 8.18535277])

In [10]:
X = np.ones(32)
iERB_Matrix = ERB_pro_matrix(ERBB, NFFT=960, mode=1) # inverse ERB convert matrix
X @ iERB_Matrix.T

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [5]:
# iERB_Matrix = ERB_pro_matrix(ERBB, NFFT=960, mode=1) # inverse ERB convert matrix
# iShane_B = Shane_B @ iERB_Matrix.T
# np.round(iShane_B,3)